# Decision Tress and Random Forest Code Along Examples

### 1. Import koniecznych pakietów oraz wstępne przygotowanie danych.

Zaczynamy od importowania pakietu findspark, aby w dalszej kolejności umożliwić import Sparka i jego klas.

In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()

Import danych z pliku CSV.

In [5]:
data = spark.read.csv('College.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [7]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

Importujemy klasę odpowiedzialną za wstępną obróbkę danych - utworzenie kolumn 'features' oraz 'label'.

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [10]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [11]:
output = assembler.transform(data)

Kolumna Private, która opisuje label poszczególnych szkół ma wartości typu string ('Yes'/'No'). Musimy użyć klasy StringIndexer amy zamienić wartości typu string na liczby.

In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [14]:
output_fixed = indexer.fit(output).transform(output)

Sprawdzmy jakie kolumny posiada DataFrame output_fixed. Ostatnie dwie kolumny to features - vektor zawierajacy wszystkie featury oraz PrivateIndex - jego wartość powstałą po zamienieniu kolumny Privte na wartości liczbowe. Pozostałe kolumny nie są nam potrzebne. Tworzymy nowy DataFrame, w którym umieszczamy tylko potrzbne nam kolumny.

In [15]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = true)



In [16]:
final_data = output_fixed.select('features', 'PrivateIndex')

Następnie dokonujemy podziału danych na zastaw treningowy i testowy. 

In [18]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

Treningowy zestaw danych:

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|       PrivateIndex|
+-------+-------------------+
|  count|                513|
|   mean| 0.2573099415204678|
| stddev|0.43757831379280476|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



Testowy zestaw danych:

In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|       PrivateIndex|
+-------+-------------------+
|  count|                264|
|   mean|0.30303030303030304|
| stddev| 0.4604410836075252|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



### 2. Tworzenie i trening klasyfikatorów

Importujemy klasy związane z drzewami decyzyjnymi.

In [21]:
from pyspark.ml.classification import (DecisionTreeClassifier, 
                                       GBTClassifier, 
                                       RandomForestClassifier)

In [22]:
from pyspark.ml import Pipeline

In [38]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features', numTrees=150)
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [39]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [40]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

### 3. Ewaluacja otrzymanych wyników

Ewaluacja przy pomocy BinaryClassificationEvaluator.

In [41]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [42]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [43]:
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))

DTC
0.936209239130435


In [44]:
print('RFC')
print(my_binary_eval.evaluate(rfc_preds))

RFC
0.9786005434782613


GBT nie ma kolumny rawPrediction, z tego względu musimy zmnienić argument domyślny dla BinaryClassificationEvaluator.

In [45]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = true)
 |-- prediction: double (nullable = true)



In [46]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [47]:
print('GBT')
print(my_binary_eval2.evaluate(gbt_preds))

GBT
0.8970108695652174


Ewaluacja przy pomocy MulticlassClassificationEvaluator.

In [48]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Określanie dokładności modelu:

In [51]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [ ]:
rfc_acc = acc_eval.evaluate(rfc)